In [1]:
# install the latest version of the openai library
!pip install openai -q --upgrade

# imports
from openai import OpenAI
import json
import time
import requests
from openai.types.beta.assistant import Assistant


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
def load_openai_key(file_path='../secrets/openAI_key') -> str:
    """
    Loads the OpenAI API key from a specified file.

    :param file_path: The path to the file containing the OpenAI API key.
    :return: The OpenAI API key, or an empty string if the file cannot be found.
    """
    try:
        with open(file_path, 'r') as file:
            openai_key = file.read().strip()  # Use strip() to remove newline characters
            print("OpenAI key loaded successfully.")
            return openai_key
    except FileNotFoundError:
        print(f"OpenAI key file not found at {file_path}.")
        return ""

def load_github_token(file_path='../secrets/github_token') -> str:
    """
    Loads the GitHub token from a specified file.

    :param file_path: The path to the file containing the GitHub token.
    :return: The GitHub token, or an empty string if the file cannot be found.
    """
    try:
        with open(file_path, 'r') as file:
            github_token = file.read().strip()  # Use strip() to remove newline characters
            print("GitHub token loaded successfully.")
            return github_token
    except FileNotFoundError:
        print(f"GitHub token file not found at {file_path}.")
        return ""


In [3]:
# API request helper functions
from typing import Dict, Any, Union, List, Optional

def remove_useful_urls(response_json: Dict[str, Any]) -> Dict[str, Any]:
    """
    Removes the 'usefulUrls' key from the API response JSON object if present.

    :param response_json: A dictionary representing the JSON response from an API.
    :return: The modified dictionary with the 'usefulUrls' key removed if it was present.
    """
    if 'usefulUrls' in response_json:
        del response_json['usefulUrls']
    return response_json


def make_api_request(endpoint_url: str, params: Dict[str, Any], github_token: str) -> Union[Dict[str, Any], str]:
    """
    Makes a POST request to the specified API endpoint with given parameters and returns the processed JSON response.
    Removes the 'usefulUrls' field from the response if it exists. In case of an error, returns an error message.

    :param endpoint_url: The URL of the API endpoint to which the request is made.
    :param params: A dictionary of parameters to be sent in the request.
    :param github_token: The GitHub token used for Authorization header.
    :return: A dictionary representing the JSON response from the API or a string containing an error message.
    """
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {github_token}",
        "Content-Type": "application/json"
    }

    response = requests.post(endpoint_url, json=params, headers=headers)

    if response.status_code == 200:
        response_json = response.json()
        response_json = remove_useful_urls(response_json)
        return response_json
    else:
        return f"Failed to communicate with the API: {response.status_code}, Reason: {response.reason}"

"""
ASK THE CODE API functions
"""

def get_repo_structure(url: str, branch: Optional[str] = None, relativePaths: Optional[List[str]] = None) -> Union[Dict[str, Any], str]:
    """
    Retrieves the structure of a GitHub repository at the specified URL, optionally filtering by branch and relative paths.

    :param url: The URL of the GitHub repository.
    :param branch: Optional; the specific branch to get the structure from.
    :param relativePaths: Optional; specific relative paths within the repository to include in the structure.
    :return: A dictionary representing the structure of the repository or an error message.
    """
    get_repo_structure_url = "https://gabriel.askthecode.ai/api/repository/structure"
    params = {
        'url': url,
        'branch': branch,
        'relativePaths': relativePaths
    }
    return make_api_request(get_repo_structure_url, params, github_token)

# get repo content
def get_repo_content(url: str, filePaths: List[str], branch: Optional[str] = None, relativePath: Optional[str] = None) -> Union[Dict[str, Any], str]:
    """
    Retrieves the content of specific files within a GitHub repository, optionally filtered by branch and a relative path.

    :param url: The URL of the GitHub repository.
    :param filePaths: The list of file paths within the repository whose content is to be retrieved.
    :param branch: Optional; the specific branch to get the content from.
    :param relativePath: Optional; the specific relative path within the repository to filter the file paths.
    :return: A dictionary representing the content of the specified files or an error message.
    """
    get_repo_content_url = "https://gabriel.askthecode.ai/api/repository/content"
    params = {
        'url': url,
        'filePaths': filePaths,
        'branch': branch,
        'relativePath': relativePath
    }
    return make_api_request(get_repo_content_url, params, github_token)
    

# get repo branches
def get_repo_branches(url: str) -> Union[Dict[str, Any], str]:
    """
    Retrieves the list of branches for the specified GitHub repository.

    :param url: The URL of the GitHub repository.
    :return: A dictionary containing the list of branches or an error message.
    """
    get_repo_branches_url = "https://gabriel.askthecode.ai/api/repository/branch/list"
    params = {'url': url}
    return make_api_request(get_repo_branches_url, params, github_token)
   
    
# Get commit history
def get_commit_history(url: str, branch: Optional[str] = None, filePath: Optional[str] = None) -> Union[Dict[str, Any], str]:
    """
    Retrieves the commit history for a specified file or branch within a GitHub repository.

    :param url: The URL of the GitHub repository.
    :param branch: Optional; the specific branch to retrieve the commit history from.
    :param filePath: Optional; the specific file path to retrieve the commit history for.
    :return: A dictionary representing the commit history or an error message.
    """
    get_commit_history_url = "https://gabriel.askthecode.ai/api/repository/commit/history"
    params = {
        'url': url,
        'branch': branch,
        'filePath': filePath
    }
    return make_api_request(get_commit_history_url, params, github_token)
    
# search repo code
def search_repo_code(url: str, searchKeywords: List[str], branch: Optional[str] = None, relativePath: Optional[str] = None, searchHitLinesCount: Optional[int] = None, skipMatchesCount: Optional[int] = None) -> Union[Dict[str, Any], str]:
    """
    Searches for specific keywords within the code of a GitHub repository, with optional filtering by branch, path, and pagination controls.

    :param url: The URL of the GitHub repository.
    :param searchKeywords: A list of keywords to search within the repository code.
    :param branch: Optional; the specific branch to search within.
    :param relativePath: Optional; the specific directory path to limit the search to.
    :param searchHitLinesCount: Optional; the number of lines to include in each search hit.
    :param skipMatchesCount: Optional; the number of matches to skip (for pagination).
    :return: A dictionary representing the search results or an error message.
    """
    search_repo_code_url = "https://gabriel.askthecode.ai/api/search/repository/code"
    params = {
        'url': url,
        'searchKeywords': searchKeywords,
        'branch': branch,
        'relativePath': relativePath,
        'searchHitLinesCount': searchHitLinesCount,
        'skipMatchesCount': skipMatchesCount
    }
    return make_api_request(search_repo_code_url, params, github_token)

# search repo commits
def search_repo_commits(url: str, searchKeywords: List[str]) -> Union[Dict[str, Any], str]:
    """
    Searches for commits in a GitHub repository based on specified keywords.

    :param url: The URL of the GitHub repository.
    :param searchKeywords: A list of keywords to search for within the commit history.
    :return: A dictionary representing the search results or an error message.
    """
    search_repo_commits_url = "https://gabriel.askthecode.ai/api/search/repository/commits"
    params = {
        'url': url,
        'searchKeywords': searchKeywords
    }
    return make_api_request(search_repo_commits_url, params, github_token)
    
def find_repos(searchKeywords: List[str], language: Optional[str] = None) -> Union[Dict[str, Any], str]:
    """
    Searches for GitHub repositories based on specified keywords and optionally filtered by programming language.

    :param searchKeywords: A list of keywords to search for repositories.
    :param language: Optional; the programming language to filter the search results by.
    :return: A dictionary representing the search results or an error message.
    """
    find_repos_url = "https://gabriel.askthecode.ai/api/search/repository"
    params = {
        'searchKeywords': searchKeywords,
        'language': language
    }
    return make_api_request(find_repos_url, params, github_token)



### Assistant API

In [4]:
def load_tools(file_path: str) -> list:
    """
    Loads tool definitions from a JSON file.

    :param file_path: The path to the JSON file containing the tools definition.
    :return: A list of tool definitions.
    """
    with open(file_path, 'r') as file:
        tools = json.load(file)
    return tools

# use
tools = load_tools('tools.json')

print(tools)
print(type(tools))

[{'type': 'function', 'function': {'name': 'get_repo_structure', 'description': 'Retrieves the Github repository file structure to analyze it and be able to query only relevant files. If the provided URL contains specific branch and directory information, prioritize using that over querying the entire repository structure.', 'parameters': {'type': 'object', 'properties': {'url': {'minLength': 1, 'type': 'string', 'description': 'Full Github repository URL provided by the user. For example: https://github.com/[owner]/[repo]/blob/[branch]/[file-path]#[additional-parameters]. The URL MUST be identical to the one, that was provided by the user, you MUST NEVER alter or truncate it. This is crucial for valid responses. You should NEVER truncate additional-parameters.'}, 'branch': {'type': 'string', 'description': 'Repository branch. Provide only if user has explicitly specified it or the previous plugin response contains it.', 'nullable': True}, 'relativePaths': {'type': 'array', 'items': {'

In [5]:
def initialize_client(api_key: str) -> OpenAI:
    """
    Initializes the OpenAI client with the given API key.

    :param api_key: The API key for authenticating requests to OpenAI.
    :return: An instance of the OpenAI client.
    """
    client = OpenAI(api_key=api_key)
    return client

from openai.types.beta.assistant import Assistant

def create_assistant(client: OpenAI, name: str, instructions: str, model: str, tools: list) -> Assistant:
    """
    Creates an assistant with the specified parameters using the provided OpenAI client.

    :param client: The OpenAI client instance.
    :param name: The name of the assistant.
    :param instructions: The instructions for the assistant.
    :param model: The model to be used by the assistant.
    :param tools: The tools to be enabled for the assistant.
    :return: The created assistant object.
    """
    assistant = client.beta.assistants.create(
        name=name,
        instructions=instructions,
        model=model,
        tools=tools
    )
    #print(assistant)
    return assistant

def retrieve_assistant(client: OpenAI, assistant_id: str) -> Assistant:
    """
    Retrieves an existing assistant by its ID using the provided OpenAI client.

    :param client: The OpenAI client instance.
    :param assistant_id: The ID of the assistant to retrieve.
    :return: The retrieved assistant object.
    """
    current_assistant = client.beta.assistants.retrieve(assistant_id)
    #print(current_assistant)
    return current_assistant



In [6]:
# Utility function to create a message and run
from typing import Tuple, Any, Union, Dict, Callable
from openai.types.beta.assistant import Assistant
from openai.types.beta.thread import Thread
from openai.types.beta.threads.run import Run


def create_message_and_run(assistant: Assistant, query: str, thread: Thread = None) -> Tuple[Run, Thread]:
    """
    Creates a message and initiates a run for a given query within a thread. If no thread is provided, a new one is created.

    :param assistant: The Assistant object to use for creating the run.
    :param query: The user's query to send to the assistant.
    :param thread: Optional; The thread object to continue the conversation. A new thread is created if not provided.
    :return: A tuple containing the Run object that was created and the Thread object used or created.
    """
    # Create a new thread if not provided
    if not thread: 
        thread = client.beta.threads.create()

    # Create a message
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=query
    )
    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )
    return run, thread

# Utility function to get details of function to be called

def get_function_details(run: Run) -> Tuple[str, Any, str]:
    """
    Extracts and prints details about the function to be called based on the run's required action.

    :param run: The Run object containing details about the required action.
    :return: A tuple containing the function name to be called, its arguments, and the function call's ID.
    """
    print("\nRun Action Required")  # Placeholder for actual run.required_action

    function_name = run.required_action.submit_tool_outputs.tool_calls[0].function.name
    arguments = run.required_action.submit_tool_outputs.tool_calls[0].function.arguments
    function_id = run.required_action.submit_tool_outputs.tool_calls[0].id

    print(f"Function Called: {function_name} with arguments: {arguments}")

    return function_name, arguments, function_id

# Utility function to submit the function response

def submit_tool_outputs(run: Run, thread: Thread, function_id: str, function_response: Any) -> Run:
    """
    Submits the output of a tool function call as part of the conversation with an assistant.

    :param run: The Run object representing the current assistant's run.
    :param thread: The Thread object where the conversation is taking place.
    :param function_id: The identifier of the function call within the assistant's run.
    :param function_response: The response from the function call to be submitted.
    :return: An updated Run object after submitting the tool outputs.
    """
    run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=[
            {
                "tool_call_id": function_id,
                "output": str(function_response),
            }
        ]
    )
    return run

def create_function_executor() -> Dict[str, Callable]:
    """
    Creates and returns a dictionary of available functions that can be executed 
    by the assistant.

    :return: A dictionary with function names as keys and callable Python functions as values.
    """
    available_functions = {
        "get_repo_structure": get_repo_structure,
        "get_repo_content": get_repo_content,
        "get_repo_branches": get_repo_branches,
        "get_commit_history": get_commit_history,
        "search_repo_code": search_repo_code,
        "search_repo_commits": search_repo_commits,
        "find_repos": find_repos
    }
    return available_functions

def execute_function_call(function_name: str, arguments: str) -> Union[Dict[str, Any], str]:
    """
    Executes a named function with provided JSON-formatted string arguments.

    :param function_name: The name of the function to be executed.
    :param arguments: A JSON-formatted string representing the arguments for the function.
    :return: The result of the function execution, which could be a dictionary or an error message.
    """
    available_functions = create_function_executor()
    function = available_functions.get(function_name, None)
    if function:
        arguments = json.loads(arguments)
        results = function(**arguments)
    else:
        results = f"Error: function {function_name} does not exist"
    return results

In [7]:
def create_new_thread(client: OpenAI) -> str:
    """
    Creates a new thread using the OpenAI client.

    :param client: The OpenAI client instance.
    :return: The ID of the newly created thread.
    """
    thread = client.beta.threads.create()
    print(f"New thread created with ID: {thread.id}")
    return thread.id

def load_thread(client: OpenAI, thread_id: str) -> dict:
    """
    Loads an existing thread by its ID.

    :param client: The OpenAI client instance.
    :param thread_id: The ID of the thread to load.
    :return: The thread object.
    """
    thread = client.beta.threads.retrieve(thread_id)
    print(f"Thread {thread_id} loaded successfully.")
    return thread

def run_chatbot(client: OpenAI, assistant: Assistant, thread_id: str = None):
    """
    Runs the main chatbot loop, allowing user interaction with the assistant.

    :param client: The OpenAI client instance.
    :param assistant: The Assistant object to use.
    :param thread_id: Optional; the ID of an existing thread to continue the conversation from.
    """
    if thread_id:
        # Load an existing thread if an ID is provided
        thread = load_thread(client, thread_id)
    else:
        # Create a new thread otherwise
        thread_id = create_new_thread(client)
        thread = load_thread(client, thread_id)  # Load the newly created thread for consistency

    while True:
        user_input = input("Please enter your query or type 'STOP' to exit: ")
        if user_input.lower() == "stop":
            break

        # Note: create_message_and_run might need adjustment if it relies on assistant_id
        # Assuming create_message_and_run is adjusted to accept an assistant object
        run, _ = create_message_and_run(assistant=assistant, query=user_input, thread=thread)

        while True:
            run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
            print("run status", run.status)

            if run.status == "requires_action":
                function_name, arguments, function_id = get_function_details(run)
                function_response = execute_function_call(function_name, arguments)
                run = submit_tool_outputs(run, thread, function_id, function_response)
                continue

            if run.status == "completed":
                messages = client.beta.threads.messages.list(thread_id=thread.id)
                latest_message = messages.data[0]
                text = latest_message.content[0].text.value
                print(text)
                break

            time.sleep(1)



In [9]:
# Full Usage:

# Load secrets
openAI_key = load_openai_key()
github_token = load_github_token()

# Initialize client
client = initialize_client(openAI_key)

# Create assistant or retrieve existing assistant
# Useage
client = initialize_client(openAI_key)
new_assistant = create_assistant(
    client=client,
    name="CodeCompass",
    instructions="You are a helpful assistant that analyzes code from github repositories and files when given a github url. You will answer questions about the structure of a repository, the content of files, or any other code-related queries.",
    model="gpt-3.5-turbo-0125",
    tools=load_tools('tools.json')
)

# Retrieve the assistant
older_assistant = retrieve_assistant(client, 'asst_gDv1PyXxMQ0GPQLYyh5KUm9C')

# Run the chatbot
run_chatbot(client, new_assistant)

OpenAI key loaded successfully.
GitHub token loaded successfully.
New thread created with ID: thread_wMbHBtor5IFD1VElI6SlAfr5
Thread thread_wMbHBtor5IFD1VElI6SlAfr5 loaded successfully.
run status in_progress
run status requires_action

Run Action Required
Function Called: get_repo_structure with arguments: {"url":"https://github.com/RecandChat/CodeCompass"}
run status in_progress
run status in_progress
run status in_progress
run status in_progress
run status in_progress
run status in_progress
run status in_progress
run status requires_action

Run Action Required
Function Called: get_repo_content with arguments: {"url":"https://github.com/RecandChat/CodeCompass","filePaths":["app.py","backend/__init__.py","codecompasslib/API/drive_operations.py","codecompasslib/clean_data.py","frontend/search.html","requirements.txt","tests/test_drive.py","LICENSE","README.md","docs/Project_Proposal.pdf","docs/Code Compass - Midterm Review.pdf","allReposCleaned.csv","codecompasslib/API/helper_functions

In [ ]:
""""
Test the API functions
"""
# Structure retrieval
test_repo_structure_url = get_repo_structure(url="https://github.com/recommenders-team/recommenders")
print("Here is the structure of the repository:")
print(test_repo_structure_url)

# Content retrieval
test_repo_content_url = get_repo_content(url="https://github.com/recommenders-team/recommenders", filePaths=[".github/.codecov.yml"])
print("Here is the content of the repository:")
print(test_repo_content_url)

# Branches retrieval
test_repo_branches_url = get_repo_branches(url="https://github.com/RecandChat/CodeCompass")
print("Here are the branches of the repository:")
print(test_repo_branches_url)

# Commit history retrieval
test_commit_history_url = get_commit_history(url="https://github.com/recommenders-team/recommenders", filePath=".devcontainer/devcontainer.json")
print("Here is the commit history of the repository:")
print(test_commit_history_url)

# Code search
search_keywords = ["Neural News Recommendation"]
test_search_repo_code_url = search_repo_code(url="https://github.com/recommenders-team/recommenders", searchKeywords=search_keywords)
print("Here is the Code search result:")
print(test_search_repo_code_url)

# Commits search
search_keywords = ["Documentation"]
test_search_repo_commits_url = search_repo_commits(url="https://github.com/recommenders-team/recommenders", searchKeywords=search_keywords)
print("Here is the Commits search result:")
print(test_search_repo_commits_url)

# find repos
search_keywords = ["recommenders"]
test_find_repos_url = find_repos(searchKeywords=search_keywords)
print("Here are the repositories found:")
print(test_find_repos_url)